In [2]:
import json
import math
import numpy as np
import pickle as pkl
import os
import time

import torch
from torch.utils.data import RandomSampler

from data_processor import DataProcessor
from trainer import Trainer
import argparse
import torchvision.models as models

In [3]:

# === DATA LOADING HELPERS =============================================================================================
# find the dataset filepaths
def get_dataset_paths(data_dir):
    paths = sorted([os.path.join(data_dir, d) for d in os.listdir(data_dir) if 'dataset' in d], reverse=True)
    return paths

# load the dataset metadata from json
def load_dataset_metadata(dataset_path):
    with open(os.path.join(dataset_path, 'metadata'), "r") as f:
        metadata = json.load(f)
    return metadata

# load dataset from file
def load_datasets(data_path, truncate):
    #data_path = '../../../datasets_val/'+data_path
    data_path = '/home/woody/iwb3/iwb3021h/datasets_val1/'+data_path
    
    train_x = np.load(os.path.join(data_path,'train_x.npy'))
    train_y = np.load(os.path.join(data_path,'train_y.npy'))
    valid_x = np.load(os.path.join(data_path,'valid_x.npy'))
    valid_y = np.load(os.path.join(data_path,'valid_y.npy'))
    test_x = np.load(os.path.join(data_path,'test_x.npy'))
    metadata = load_dataset_metadata(data_path)

    if truncate:
        train_x = train_x[:64]
        train_y = train_y[:64]
        valid_x = valid_x[:64]
        valid_y = valid_y[:64]
        test_x = test_x[:64]

    return (train_x, train_y), \
           (valid_x, valid_y), \
           (test_x), metadata


# === TIME COUNTERs ====================================================================================================
def div_remainder(n, interval):
    # finds divisor and remainder given some n/interval
    factor = math.floor(n / interval)
    remainder = int(n - (factor * interval))
    return factor, remainder


def show_time(seconds):
    # show amount of time as human readable
    if seconds < 60:
        return "{:.2f}s".format(seconds)
    elif seconds < (60 * 60):
        minutes, seconds = div_remainder(seconds, 60)
        return "{}m,{}s".format(minutes, seconds)
    else:
        hours, seconds = div_remainder(seconds, 60 * 60)
        minutes, seconds = div_remainder(seconds, 60)
        return "{}h,{}m,{}s".format(hours, minutes, seconds)


# keep a counter of available time
class Clock:
    def __init__(self, time_available):
        self.start_time =  time.time()
        self.total_time = time_available

    def check(self):
        return self.total_time + self.start_time - time.time()


# === MODEL ANALYSIS ===================================================================================================
def general_num_params(model):
    # return number of differential parameters of input model
    return sum([np.prod(p.size()) for p in filter(lambda p: p.requires_grad, model.parameters())])


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')

In [5]:
device

device(type='cuda')

In [9]:
# === MAIN =============================================================================================================
# the available runtime will change at various stages of the competition, but feel free to change for local tests
# note, this is approximate, your runtime will be controlled externally by our server
total_runtime_hours = 30
total_runtime_seconds = total_runtime_hours * 60 * 60


dataset="Voxel"
#MultNIST, Language, Gutenberg, CIFARTile, Chesseract, AddNIST, 
#CIFAR10, ImageNet16-120; Sudoku, Voxel, 
#select_augment="Model"
select_augment="Proxy"
model_name="EfficientNet_b0"
# ResNet18, MobileNetV3_large; ,RegNetY_400MF, EfficientNet_b0
seed=1

runclock = Clock(total_runtime_seconds)


# load and display data info
(train_x, train_y), (valid_x, valid_y), (test_x), metadata = load_datasets(dataset, truncate=False)
metadata['time_remaining'] = runclock.check()
start_time = time.time()

print("Metadata:")
[print("   - {:<20}: {}".format(k, v)) for k,v in metadata.items()]

# perform data processing/augmentation/etc using your DataProcessor
print("\n=== Processing Data ===")
print("  Allotted compute time remaining: ~{}".format(show_time(runclock.check())))

#### Select model
import torch.nn as nn
metadata["model_name"]=model_name
if model_name=="ResNet18":
    model = models.resnet18(weights=None)
    new_conv1 = torch.nn.Conv2d(in_channels=metadata["input_shape"][1], 
                              out_channels=model.conv1.out_channels, 
                              kernel_size=model.conv1.kernel_size, 
                              stride=model.conv1.stride, 
                              padding=model.conv1.padding, 
                              bias=model.conv1.bias)
    # Replace the first convolutional layer
    model.conv1 = new_conv1
    model.fc = torch.nn.Linear(512, metadata['num_classes'])
    model.to(device)
####
if model_name == "RegNetY_400MF":
        # Load a predefined RegNet architecture
        model = models.regnet_y_400mf(weights=None)  # Using no pre-trained weights

        # Adjust the first convolution layer for the specific input channels
        new_stem_conv = torch.nn.Conv2d(
            in_channels=metadata["input_shape"][1],  # Dynamic input channels
            out_channels=model.stem[0].out_channels,  # Preserve original out channels
            kernel_size=model.stem[0].kernel_size,
            stride=model.stem[0].stride,
            padding=model.stem[0].padding,
            bias=model.stem[0].bias
        )
        # Replace the stem's initial convolutional layer
        model.stem[0] = new_stem_conv

        # Replace the fully connected layer with a dynamic output layer
        model.fc = torch.nn.Linear(model.fc.in_features, metadata['num_classes'])

        # Move the model to the specified device
        model.to(device)

if model_name == "VGG19":
    model = models.vgg19(weights=None)
    # Modify the first convolutional layer
    model.features[0] = torch.nn.Conv2d(
        in_channels=metadata["input_shape"][1],
        out_channels=model.features[0].out_channels,
        kernel_size=model.features[0].kernel_size,
        stride=model.features[0].stride,
        padding=model.features[0].padding
    )

    # Add a global average pooling before the classifier
    model.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))

    # Modify the classification layer
    model.classifier[-1] = torch.nn.Linear(model.classifier[-1].in_features, metadata['num_classes'])

    # Move model to device
    model.to(device)


if model_name == "MobileNetV3_large":
    model = models.mobilenet_v3_large(weights=None)
    # Modify the first convolutional layer
    model.features[0][0] = torch.nn.Conv2d(
        in_channels=metadata["input_shape"][1],
        out_channels=model.features[0][0].out_channels,
        kernel_size=model.features[0][0].kernel_size,
        stride=model.features[0][0].stride,
        padding=model.features[0][0].padding,
        bias=model.features[0][0].bias
    )
    # Modify the classification layer
    model.classifier[-1] = torch.nn.Linear(model.classifier[-1].in_features, metadata['num_classes'])
    model.to(device)


if model_name == "EfficientNet_b0":
    model = models.efficientnet_b0(weights=None)
    # Modify the first convolutional layer
    model.features[0][0] = torch.nn.Conv2d(
        in_channels=metadata["input_shape"][1],
        out_channels=model.features[0][0].out_channels,
        kernel_size=model.features[0][0].kernel_size,
        stride=model.features[0][0].stride,
        padding=model.features[0][0].padding,
        bias=model.features[0][0].bias
    )
    # Modify the classification layer
    model.classifier[-1] = torch.nn.Linear(model.classifier[-1].in_features, metadata['num_classes'])
    model.to(device)

####
    
class ModifiedResNet18(models.ResNet):
    def __init__(self, metadata):
        super().__init__(block=models.resnet.BasicBlock, layers=[2, 2, 2, 2])
        
        # Modify the first convolutional layer
        self.conv1 = nn.Conv2d(
            in_channels=metadata["input_shape"][1],
            out_channels=self.conv1.out_channels,
            kernel_size=self.conv1.kernel_size,
            stride=self.conv1.stride,
            padding=self.conv1.padding,
            bias=self.conv1.bias,
        )
        
        # Modify the final fully connected layer
        self.fc = nn.Linear(512, metadata['num_classes'])
        
    def forward_before_global_avg_pool(self, x):
        # Extract all layers before the global average pooling
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        return x  # Return feature map before pooling


#model = ModifiedResNet18(metadata)
#model.to(device)

#####

data_processor = DataProcessor(train_x, train_y, valid_x, valid_y, test_x, metadata, select_augment, model)
train_loader, valid_loader, test_loader = data_processor.process()
metadata['time_remaining'] = runclock.check()

# check that the test_loader is configured correctly
#assert_string = "Test Dataloader is {}, this will break evaluation. Please fix this in your DataProcessor init."
#assert not isinstance(test_loader.sampler, RandomSampler), assert_string.format("shuffling")
#assert not test_loader.drop_last, assert_string.format("dropping last batch")

# search for best model using your NAS algorithm
#print("\n=== Performing NAS ===")
#print("  Allotted compute time remaining: ~{}".format(show_time(runclock.check())))



#model_params = int(general_num_params(model))
#metadata['time_remaining'] = runclock.check()

# train model using your Trainer
#print("\n=== Training ===")
#print("  Allotted compute time remaining: ~{}".format(show_time(runclock.check())))
#device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
#trainer = Trainer(model, device, train_loader, valid_loader, metadata)
#trained_model = trainer.train()

# submit predictions to file
#print("\n=== Predicting ===")
#print("  Allotted compute time remaining: ~{}".format(show_time(runclock.check())))
#predictions = trainer.predict(test_loader)
#run_data = {'Runtime': float(np.round(time.time()-start_time, 2)), 'Params': model_params}
#with open("predictions/{}_stats.pkl".format(metadata['codename']), "wb") as f:
#    pkl.dump(run_data, f)
#np.save('predictions/{}.npy'.format(metadata['codename']), predictions)


ic| self.select_augment: 'Proxy'


Metadata:
   - input_shape         : [50000, 20, 20, 20]
   - codename            : Volga
   - benchmark           : 71.35
   - num_classes         : 7
   - time_remaining      : 107980.68109369278

=== Processing Data ===
  Allotted compute time remaining: ~29h,59m,40s


ic| self.x.shape: torch.Size([50000, 20, 20, 20])
ic| unique_values: array([0., 1.], dtype=float32)
ic| C: 20
ic| H: 20
ic| PH: 2
/apps/jupyterhub/jh3.1.1-py3.11/envs/pytorch-2.2.0/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(
ic| '#############'
ic| aug: 0
ic| train_loader.dataset.transform.transforms: [Normalize(mean=tensor([0.0724, 0.0832, 0.0932, 0.1020, 0.1097, 0.1162, 0.1213, 0.1250, 0.1275,
                                                       0.1288, 0.1289, 0.1274, 0.1250, 0.1211, 0.1157, 0.1091, 0.1012, 0.0924,
                                                       0.0824, 0.0716]), std=tensor([0.2592, 0.2762, 0.2907, 0.3026, 0.3125, 0.3204, 0.3264, 0.3307, 0.3336,
                                                       0.3350, 0.3351, 0.3334, 0.3308,

# Read training results for augmentations

In [10]:
def results_to_df(path):
    data = []
    # Open the text file
    with open(path, 'r') as file:
        lines = file.readlines()
        # Initialize an empty dictionary to store data for each block
        block_data = {}
        for line in lines:
            # If the line contains dashes, it indicates the end of a block
            if '-------------------------' in line:
                # If block_data is not empty, add it to the list of data dictionaries
                if block_data:
                    data.append(block_data)
                    # Reset block_data for the next block
                    block_data = {}
            elif 'best_acc' in line:
                continue
            else:
                # Split the line by ':'
                #print(line)
                key, value = line.strip().split(': ')
                # Store the key-value pair in the block_data dictionary
                block_data[key] = value

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)

    # Convert columns to appropriate data types if needed
    df['epoch'] = df['epoch'].astype(int)
    df['lr'] = df['lr'].astype(float)
    df['train_acc'] = df['train_acc'].astype(float)
    df['train_loss'] = df['train_loss'].astype(float)
    df['test_acc'] = df['test_acc'].astype(float)
    df['test_acc_top5'] = df['test_acc_top5'].astype(float)
    df['test_loss'] = df['test_loss'].astype(float)
    df['epoch_time'] = df['epoch_time'].astype(float)

    
    return df


In [11]:
correlations=[]

In [40]:
import pandas as pd
dataset="Gutenberg"
#Adaline, Caitie, Chester, CIFAR10, Gutenberg, in16, LaMelo, Mateo, Sokoto, Volga
#MobileNetV3_large, EfficientNet_b0,  RegNetY_400MF
model_name="EfficientNet_b0"
for dataset in ["Adaline", "Caitie", "Chester", "CIFAR10", "Gutenberg", "in16", "LaMelo", "Mateo", "Sokoto", "Volga"]:
    rows=[]
    for aug in range(23):
        try:
            df_scores=results_to_df(f"/home/woody/iwb3/iwb3021h/augmentations_test/{model_name}/{dataset}/aug_{aug}/worklog.txt")

            row_max = df_scores.loc[[df_scores['test_acc'].idxmax()]]
            row_max=row_max.assign(aug=aug)
            rows.append(row_max[["train_acc","test_acc","test_loss", "aug"]])
        except:
            print(aug)        
        total_scores_df=pd.concat(rows)

    #################
    measures=["epe_nas","nwot","plain","snip", "fisher","jacob_cov", "grad_norm","grasp"]
    zcosts=pd.read_csv(f"/home/woody/iwb3/iwb3021h/augmentations_test/{model_name}/{dataset}/ranks_{dataset}.csv")#, index_col=0)
    try: 
        zcosts=zcosts[measures+["aug"]]#"synflow","l2_norm"
    except:
        measures=["epe_nas","nwot","plain","snip", "fisher","jacob_cov", "grad_norm"]
        zcosts=zcosts[measures+["aug"]]#"synflow","l2_norm"
    ####################

    zcosts=pd.merge(total_scores_df, zcosts, on="aug", how="left").sort_values(by="test_acc", ascending=False)

    # Check for columns with complex values
    for col in zcosts.select_dtypes(include=['O']).columns:
        # Convert to complex and take only the real part
        zcosts[col] = pd.to_numeric(zcosts[col], errors='coerce', downcast=None).apply(
            lambda x: x.real if pd.notnull(x) else x
        )

    zcosts.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Fill NaN (previously inf) with the column means
    zcosts.fillna(zcosts.mean(), inplace=True)

    #Correlation
    spearman_corr = zcosts.corr(method='spearman')
    spearman_corr.to_csv(f"/home/woody/iwb3/iwb3021h/augmentations_test/{model_name}/{dataset}/corr_{dataset}.csv")

    spearman_corr=spearman_corr[["test_acc","aug","test_loss"]].iloc[4:]
    spearman_corr["model"]=model_name
    spearman_corr["dataset"]=dataset
    correlations.append(spearman_corr)
    #########################
    #Scaling
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    zcosts[measures] = scaler.fit_transform(zcosts[measures])
    #########################################################
    for measure in measures:
        zcosts[f'rank_{measure}'] = zcosts[f'{measure}'].rank(ascending=False, method='dense')
    zcosts["fisher_jacob"]=zcosts[f'fisher']+zcosts[f'jacob_cov']
    zcosts[f'rank_fisher_jacob'] = zcosts["fisher_jacob"].rank(ascending=False, method='dense')
    #zcosts[f'rank_fisher_jacob'] = (zcosts[f'fisher']+zcosts[f'jacob_cov']).rank(ascending=False, method='dense')
    zcosts["model"]=model_name
    zcosts.to_csv(f"/home/woody/iwb3/iwb3021h/augmentations_test/{model_name}/{dataset}/zcosts_{dataset}.csv")


/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


1
2
3
4
5
6
7
22


/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.

1
2
3
4
5
6
7
22


/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/sklearn/utils/_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [43]:
zcosts[["test_acc", "fisher_jacob"]]

,test_acc,fisher_jacob
1,72.78,0.507096
4,72.65,0.968526
5,72.33,1.218252
2,72.14,1.325703
6,71.97,0.823424
7,71.96,1.153773
13,70.86,1.165185
11,70.63,0.966442
14,70.40,0.790863
3,69.24,1.389937


In [37]:
pd.concat(correlations)

,index,train_acc,test_acc,test_loss,aug,epe_nas,nwot,plain,snip,fisher,jacob_cov,grad_norm,dataset
1,test_acc,0.290514,1.0,-0.952736,-0.631423,0.236166,NaN,-0.037549,0.268775,0.245059,-0.339921,0.333992,Adaline
1,test_acc,0.363636,1.0,-0.972813,-0.503953,-0.055336,NaN,0.377470,0.156126,-0.206522,0.009881,0.128458,Caitie
1,test_acc,0.675000,1.0,-0.956926,-0.335714,0.200000,NaN,0.214286,0.542857,0.246429,-0.457143,-0.060714,Chester
1,test_acc,-0.627470,1.0,-0.989125,-0.142292,-0.215415,NaN,-0.063241,0.354743,0.204545,-0.195652,0.346838,CIFAR10
1,test_acc,0.874228,1.0,-0.851510,-0.127996,-0.177910,NaN,0.187299,0.078577,0.071658,-0.438349,-0.083519,Gutenberg
1,test_acc,0.727947,1.0,-0.987887,-0.473320,0.110672,NaN,0.092885,0.313241,0.115613,-0.296443,0.388340,in16
1,test_acc,0.316284,1.0,-0.872618,-0.092414,-0.324685,NaN,0.225846,0.422041,0.307388,-0.410180,0.364220,LaMelo
1,test_acc,0.517787,1.0,-0.965159,-0.679842,-0.196640,NaN,0.011858,0.427866,0.318182,-0.615613,0.591897,Mateo
1,test_acc,0.207734,1.0,0.240911,-0.048539,0.142150,NaN,0.038138,0.437842,0.206043,-0.013868,-0.174840,Sokoto
1,test_acc,-0.021429,1.0,-0.985626,-0.392857,-0.582143,NaN,0.125000,0.217857,0.260714,-0.614286,0.525000,Volga


In [9]:
pd.concat(correlations)

,test_acc,aug,test_loss,model,dataset
epe_nas,-0.057312,0.195652,0.030161,RegNetY_400MF,Adaline
nwot,-0.315217,0.595850,0.282324,RegNetY_400MF,Adaline
plain,0.207510,-0.054348,-0.289246,RegNetY_400MF,Adaline
snip,-0.455534,0.288538,0.456860,RegNetY_400MF,Adaline
fisher,-0.629447,0.354743,0.636836,RegNetY_400MF,Adaline
jacob_cov,-0.223320,0.652174,0.208653,RegNetY_400MF,Adaline
grad_norm,-0.442688,0.257905,0.439061,RegNetY_400MF,Adaline


In [9]:
pd.concat(correlations)

,test_acc,aug,test_loss,model,dataset
epe_nas,-0.009881,0.077075,0.033284,RegNetY_400MF,Sokoto
nwot,0.261428,0.236719,-0.113542,RegNetY_400MF,Sokoto
plain,0.001976,-0.529644,0.120220,RegNetY_400MF,Sokoto
snip,0.051383,0.117589,-0.351719,RegNetY_400MF,Sokoto
fisher,0.055336,0.172925,-0.356687,RegNetY_400MF,Sokoto
jacob_cov,-0.325099,-0.121542,0.349732,RegNetY_400MF,Sokoto
grad_norm,0.072134,0.161067,-0.368113,RegNetY_400MF,Sokoto


In [17]:
from scipy.stats import spearmanr
import itertools

column_combinations = list(itertools.combinations(measures, 2))


for dataset in ["Adaline", "Caitie", "Chester", "CIFAR10", "Gutenberg", "in16", "LaMelo", "Mateo", "Sokoto", "Volga"]:
    results = []

    # Iterate and compute the Spearman correlation for each combination
    for col1, col2 in column_combinations:
        combined_values = zcosts[col1] - zcosts[col2]  # Combine columns by subtracting their values
        corr, _ = spearmanr(zcosts["test_acc"], combined_values)

        # Append results to the list
        results.append({"col1": col1, "col2": col2, "corr": corr})

    # Convert results to a DataFrame and save as CSV
    results_df_neg = pd.DataFrame(results)
    results_df_neg.to_csv(f"/home/woody/iwb3/iwb3021h/augmentations_test/{model_name}/{dataset}/comb_corr_{dataset}_neg.csv", index=False)


    results = []

    # Iterate and compute the Spearman correlation for each combination
    for col1, col2 in column_combinations:
        combined_values = zcosts[col1] + zcosts[col2]  # Combine columns by subtracting their values
        corr, _ = spearmanr(zcosts["test_acc"], combined_values)

        # Append results to the list
        results.append({"col1": col1, "col2": col2, "corr": corr})

    # Convert results to a DataFrame and save as CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"/home/woody/iwb3/iwb3021h/augmentations_test/{model_name}/{dataset}/comb_corr_{dataset}_pos.csv", index=False)

In [244]:
df

,index,train_acc,test_acc,test_loss,aug,epe_nas,nwot,plain,snip,fisher,...,grad_norm,rank_epe_nas,rank_nwot,rank_plain,rank_snip,rank_fisher,rank_jacob_cov,rank_grad_norm,model,pos
0,2,66.74,67.91,0.74,9,1.373022e+06,5227.063889,-0.010614,21.345419,0.000279,...,6.842886,12.0,12.0,6.0,14.0,12.0,14.0,14.0,MobileNetV3_large,1.0
1,5,61.83,64.46,0.82,12,1.380295e+06,5120.348988,-0.030193,22.489124,0.000258,...,7.516641,9.0,14.0,15.0,4.0,14.0,11.0,5.0,MobileNetV3_large,2.0
2,1,64.04,63.94,0.81,8,1.365543e+06,5256.341822,-0.021642,22.050383,0.000321,...,6.945489,14.0,11.0,12.0,10.0,9.0,13.0,13.0,MobileNetV3_large,3.0
3,14,60.76,63.45,0.86,21,1.486250e+06,5628.782053,-0.017527,21.932199,0.000332,...,7.192066,6.0,5.0,10.0,12.0,6.0,5.0,8.0,MobileNetV3_large,4.0
4,0,67.67,63.03,0.85,0,1.375757e+06,5200.191685,-0.006991,21.727465,0.000333,...,7.003474,10.0,13.0,4.0,13.0,5.0,12.0,12.0,MobileNetV3_large,5.0
5,9,64.30,61.78,0.87,16,1.512495e+06,5692.449709,-0.007924,22.023966,0.000386,...,7.500123,2.0,2.0,5.0,11.0,1.0,2.0,6.0,MobileNetV3_large,6.0
6,6,58.27,61.42,0.88,13,1.371341e+06,4856.640451,-0.025797,22.767122,0.000325,...,7.814985,13.0,15.0,14.0,3.0,8.0,9.0,1.0,MobileNetV3_large,7.0
7,11,62.71,60.87,0.88,18,1.489730e+06,5627.411557,-0.017340,21.251942,0.000258,...,7.054242,3.0,6.0,9.0,15.0,15.0,7.0,11.0,MobileNetV3_large,8.0
8,3,59.15,60.25,0.92,10,1.363266e+06,5263.432441,0.018137,22.156715,0.000274,...,6.657001,15.0,10.0,1.0,8.0,13.0,15.0,15.0,MobileNetV3_large,9.0
9,13,57.42,59.63,0.92,20,1.488481e+06,5623.401090,0.001243,22.331259,0.000289,...,7.694211,4.0,7.0,2.0,6.0,11.0,6.0,3.0,MobileNetV3_large,10.0


In [24]:
#Adaline, Caitie, Chester, CIFAR10, Gutenberg, in16, LaMelo, Mateo, Sokoto, Volga
#MobileNetV3_large, EfficientNet_b0, ResNet18, RegNetY_400MF
import pandas as pd
model_name="EfficientNet_b0"
#dataset="Adaline"
import numpy as np
dataset_results={}
dataset_autoaugment={}
dataset_noaug={}
dataset_randaug={}
correlations=[]
correlations_pos=[]

for dataset in ["Adaline", "Caitie", "Chester", "CIFAR10", "Gutenberg", "in16", "LaMelo", "Mateo", "Sokoto", "Volga"]:
    df=pd.read_csv(f"/home/woody/iwb3/iwb3021h/augmentations_test/{model_name}/{dataset}/zcosts_{dataset}.csv", index_col=0).reset_index()
    corr=pd.read_csv(f"/home/woody/iwb3/iwb3021h/augmentations_test/{model_name}/{dataset}/corr_{dataset}.csv", index_col=0).reset_index()
    corr=corr[corr["index"]=="test_acc"]
    corr["dataset"]=dataset
    correlations.append(corr)
    
    corr_pos=pd.read_csv(f"/home/woody/iwb3/iwb3021h/augmentations_test/{model_name}/{dataset}/comb_corr_{dataset}_pos.csv", index_col=0).reset_index()
    corr_pos["dataset"]=dataset
    correlations_pos.append(corr_pos)
    
    df[f"pos"]=df["test_acc"].rank(ascending=False, method='dense')
    dataset_results[dataset]={}
    try:
        dataset_autoaugment[dataset]=df[df.aug==22].pos.values[0]
    except:
        dataset_autoaugment[dataset]=np.NaN
    try:
        dataset_noaug[dataset]=df[df.aug==0].pos.values[0]
    except:
        dataset_noaug[dataset]=np.NaN
    try:
        dataset_randaug[dataset]=df[df.aug==1].pos.values[0]
    except:
        dataset_randaug[dataset]=np.NaN
    for metric in ["epe_nas","nwot","plain","snip", "fisher","jacob_cov", "grad_norm", "fisher_jacob"]:#"grasp":
        #df.sort_values(by=metric,ascending=False)[["test_acc",metric,f"rank_{metric}","pos"]]
        try:
            score=df.sort_values(by=metric,ascending=True)[["test_acc",metric,f"rank_{metric}","pos"]].head(1).pos.values[0]
            dataset_results[dataset][metric]=score

        except:
            print(metric)

In [11]:
pd.concat(correlations_pos).groupby(["col1","col2"])["corr"].mean()

col1       col2     
epe_nas    fisher      -0.342857
           grad_norm   -0.157143
           jacob_cov   -0.560714
           nwot        -0.567857
           plain       -0.439286
           snip        -0.235714
fisher     grad_norm    0.307143
           jacob_cov   -0.432143
jacob_cov  grad_norm   -0.285714
nwot       fisher      -0.357143
           grad_norm   -0.207143
           jacob_cov   -0.653571
           plain       -0.428571
           snip        -0.228571
plain      fisher       0.150000
           grad_norm    0.303571
           jacob_cov   -0.446429
           snip         0.207143
snip       fisher       0.278571
           grad_norm    0.410714
           jacob_cov   -0.335714
Name: corr, dtype: float64

In [25]:
results=[]
results.append(pd.DataFrame(dataset_results))
results.append(pd.DataFrame(dataset_autoaugment, index=["AutoAugment"]))
results.append(pd.DataFrame(dataset_noaug, index=["NoAugmentation"]))
results.append(pd.DataFrame(dataset_randaug, index=["RandAugment"]))

In [26]:
pd.concat(results)

,Adaline,Caitie,Chester,CIFAR10,Gutenberg,in16,LaMelo,Mateo,Sokoto,Volga
epe_nas,15.0,16.0,5.0,16.0,1.0,8.0,12.0,2.0,15.0,5.0
nwot,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
plain,1.0,21.0,14.0,6.0,7.0,18.0,12.0,13.0,4.0,12.0
snip,14.0,3.0,11.0,23.0,20.0,23.0,19.0,17.0,17.0,15.0
fisher,4.0,1.0,13.0,23.0,20.0,21.0,14.0,21.0,17.0,9.0
jacob_cov,9.0,16.0,7.0,9.0,19.0,7.0,7.0,10.0,4.0,5.0
grad_norm,18.0,1.0,5.0,21.0,16.0,23.0,18.0,8.0,10.0,8.0
fisher_jacob,9.0,16.0,7.0,5.0,6.0,7.0,7.0,10.0,3.0,1.0
AutoAugment,15.0,16.0,NaN,9.0,7.0,22.0,7.0,11.0,11.0,NaN
NoAugmentation,11.0,15.0,9.0,22.0,5.0,4.0,8.0,6.0,14.0,12.0


In [14]:
pd.concat(results).mean(axis=1)

epe_nas           11.500
nwot               9.500
plain             12.800
snip               6.700
fisher             7.700
jacob_cov          8.200
grad_norm          8.300
fisher_jacob       7.500
AutoAugment        8.250
NoAugmentation    15.800
RandAugment        6.375
dtype: float64

In [40]:
pd.concat(results)

,Adaline,Caitie,Chester,CIFAR10,Gutenberg,in16,LaMelo,Mateo,Sokoto,Volga
epe_nas,10.0,8.0,7.0,16.0,19.0,2.0,12.0,11.0,6.0,6.0
nwot,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
plain,12.0,12.0,5.0,22.0,14.0,23.0,6.0,12.0,4.0,3.0
snip,1.0,3.0,6.0,3.0,11.0,14.0,15.0,13.0,10.0,8.0
fisher,14.0,3.0,8.0,16.0,16.0,10.0,21.0,6.0,14.0,8.0
jacob_cov,6.0,16.0,9.0,5.0,10.0,14.0,6.0,10.0,3.0,5.0
grad_norm,5.0,22.0,11.0,16.0,9.0,15.0,3.0,15.0,10.0,8.0
fisher_jacob,9.0,10.0,7.0,5.0,10.0,14.0,9.0,10.0,3.0,5.0
AutoAugment,15.0,16.0,NaN,9.0,7.0,22.0,7.0,11.0,11.0,NaN
NoAugmentation,11.0,15.0,9.0,22.0,5.0,4.0,8.0,6.0,14.0,12.0


In [24]:
pd.concat(results)

,Adaline,Caitie,Chester,CIFAR10,Gutenberg,in16,LaMelo,Mateo,Sokoto,Volga
epe_nas,15.0,8.0,7.0,16.0,19.0,2.0,12.0,11.0,6.0,6.0
nwot,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
plain,15.0,12.0,5.0,22.0,14.0,23.0,6.0,12.0,4.0,3.0
snip,21.0,3.0,6.0,3.0,11.0,14.0,15.0,13.0,10.0,8.0
fisher,23.0,3.0,8.0,16.0,16.0,10.0,21.0,6.0,14.0,8.0
jacob_cov,7.0,16.0,9.0,5.0,10.0,14.0,6.0,10.0,3.0,5.0
grad_norm,13.0,22.0,11.0,16.0,9.0,15.0,3.0,15.0,10.0,8.0
fisher_jacob,7.0,10.0,7.0,5.0,10.0,14.0,9.0,10.0,3.0,5.0
AutoAugment,15.0,16.0,NaN,9.0,7.0,22.0,7.0,11.0,11.0,NaN
NoAugmentation,11.0,15.0,9.0,22.0,5.0,4.0,8.0,6.0,14.0,12.0


In [12]:
pd.concat(results)

,Adaline,Caitie,Chester,CIFAR10,Gutenberg,in16,LaMelo,Mateo,Sokoto,Volga
epe_nas,10.0,2.0,1.0,20.0,9.0,4.0,19.0,16.0,7.0,6.0
nwot,19.0,9.0,1.0,8.0,14.0,5.0,4.0,6.0,6.0,2.0
plain,15.0,13.0,6.0,15.0,4.0,5.0,4.0,11.0,13.0,4.0
snip,20.0,7.0,3.0,12.0,7.0,17.0,9.0,4.0,15.0,10.0
fisher,10.0,8.0,3.0,5.0,7.0,3.0,9.0,4.0,15.0,9.0
jacob_cov,8.0,21.0,1.0,8.0,9.0,1.0,11.0,6.0,6.0,1.0
grad_norm,10.0,7.0,3.0,12.0,7.0,17.0,9.0,4.0,15.0,10.0
fisher_jacob,10.0,9.0,3.0,8.0,7.0,1.0,11.0,6.0,6.0,2.0
AutoAugment,6.0,11.0,NaN,6.0,9.0,10.0,11.0,7.0,6.0,NaN
NoAugmentation,16.0,16.0,14.0,23.0,16.0,16.0,15.0,16.0,11.0,15.0


In [16]:
pd.concat(results)

,Adaline,Caitie,Chester,CIFAR10,Gutenberg,in16,LaMelo,Mateo,Sokoto,Volga
epe_nas,9.0,2.0,1.0,20.0,9.0,4.0,19.0,16.0,7.0,6.0
nwot,16.0,9.0,1.0,8.0,14.0,5.0,4.0,6.0,6.0,2.0
plain,15.0,13.0,6.0,15.0,4.0,5.0,4.0,11.0,13.0,4.0
snip,5.0,7.0,3.0,12.0,7.0,17.0,9.0,4.0,15.0,10.0
fisher,5.0,8.0,3.0,5.0,7.0,3.0,9.0,4.0,15.0,9.0
jacob_cov,16.0,21.0,1.0,8.0,9.0,1.0,11.0,6.0,6.0,1.0
grad_norm,5.0,7.0,3.0,12.0,7.0,17.0,9.0,4.0,15.0,10.0
fisher_jacob,5.0,9.0,3.0,8.0,7.0,1.0,11.0,6.0,6.0,2.0
AutoAugment,6.0,11.0,NaN,6.0,9.0,10.0,11.0,7.0,6.0,NaN
NoAugmentation,16.0,16.0,14.0,23.0,16.0,16.0,15.0,16.0,11.0,15.0


In [12]:
pd.concat(results)

,Adaline,Caitie,Chester,CIFAR10,Gutenberg,in16,LaMelo,Mateo,Sokoto,Volga
epe_nas,8.0,2.0,1.0,20.0,9.0,4.0,19.0,16.0,7.0,6.0
nwot,16.0,9.0,1.0,8.0,14.0,5.0,4.0,6.0,6.0,2.0
plain,13.0,13.0,6.0,15.0,4.0,5.0,4.0,11.0,13.0,4.0
snip,6.0,7.0,3.0,12.0,7.0,17.0,9.0,4.0,15.0,10.0
fisher,6.0,8.0,3.0,5.0,7.0,3.0,9.0,4.0,15.0,9.0
jacob_cov,7.0,21.0,1.0,8.0,9.0,1.0,11.0,6.0,6.0,1.0
grad_norm,6.0,7.0,3.0,12.0,7.0,17.0,9.0,4.0,15.0,10.0
fisher_jacob,7.0,9.0,3.0,8.0,7.0,1.0,11.0,6.0,6.0,2.0
AutoAugment,6.0,11.0,NaN,6.0,9.0,10.0,11.0,7.0,6.0,NaN
NoAugmentation,16.0,16.0,14.0,23.0,16.0,16.0,15.0,16.0,11.0,15.0


In [13]:
pd.concat(results)

,Adaline,Caitie,Chester,CIFAR10,Gutenberg,in16,LaMelo,Mateo,Sokoto,Volga
epe_nas,19.0,2.0,1.0,20.0,9.0,4.0,19.0,16.0,7.0,6.0
nwot,4.0,9.0,1.0,8.0,14.0,5.0,4.0,6.0,6.0,2.0
plain,3.0,13.0,6.0,15.0,4.0,5.0,4.0,11.0,13.0,4.0
snip,14.0,7.0,3.0,12.0,7.0,17.0,9.0,4.0,15.0,10.0
fisher,14.0,8.0,3.0,5.0,7.0,3.0,9.0,4.0,15.0,9.0
jacob_cov,16.0,21.0,1.0,8.0,9.0,1.0,11.0,6.0,6.0,1.0
grad_norm,14.0,7.0,3.0,12.0,7.0,17.0,9.0,4.0,15.0,10.0
fisher_jacob,3.0,9.0,3.0,8.0,7.0,1.0,11.0,6.0,6.0,2.0
AutoAugment,6.0,11.0,NaN,6.0,9.0,10.0,11.0,7.0,6.0,NaN
NoAugmentation,16.0,16.0,14.0,23.0,16.0,16.0,15.0,16.0,11.0,15.0


In [13]:
pd.concat(results).mean(axis=1)

epe_nas            9.100
nwot               8.800
plain             10.400
snip              11.100
fisher             8.000
jacob_cov          5.800
grad_norm         11.100
fisher_jacob       6.700
AutoAugment        8.250
NoAugmentation    15.800
RandAugment        6.375
dtype: float64

In [227]:
pd.concat(results).mean(axis=1)

epe_nas           10.00
nwot               1.00
plain             12.60
snip              10.40
fisher            10.10
jacob_cov         14.70
grad_norm         11.00
AutoAugment       12.25
NoAugmentation    10.60
RandAugment        7.50
dtype: float64

In [27]:
#pd.concat(results).to_csv("EfficientNet_b0_pos.csv")

In [30]:
pd.read_csv("RegNetY_400MF_pos.csv", index_col=0).mean(axis=1)

epe_nas           11.100
nwot              10.900
plain             13.600
snip               7.600
fisher             8.600
jacob_cov          8.200
grad_norm          9.300
fisher_jacob       5.900
AutoAugment        8.250
NoAugmentation    15.800
RandAugment        6.375
dtype: float64

In [32]:
pd.read_csv("EfficientNet_b0_pos.csv", index_col=0).mean(axis=1)

epe_nas            9.50
nwot               1.00
plain             10.80
snip              16.20
fisher            14.30
jacob_cov          9.30
grad_norm         12.80
fisher_jacob       7.10
AutoAugment       12.25
NoAugmentation    10.60
RandAugment        7.50
dtype: float64

In [36]:
pd.read_csv("ResNet18_pos.csv", index_col=0).mean(axis=1)

epe_nas           10.500
nwot               8.100
plain              5.000
snip               9.000
fisher             8.000
jacob_cov          7.700
grad_norm          8.600
fisher_jacob       7.500
AutoAugment       11.000
NoAugmentation    13.700
RandAugment        5.375
dtype: float64

In [29]:
pd.read_csv("correlations_RegNetY_400MF.csv", index_col=0).reset_index().groupby("index")["test_acc"].mean()

index
epe_nas     -0.209635
fisher      -0.236562
grad_norm   -0.123767
grasp        0.112866
jacob_cov   -0.362412
nwot        -0.250534
plain       -0.006321
snip        -0.163792
Name: test_acc, dtype: float64

In [30]:
pd.read_csv("correlations_EfficientNet_b0.csv", index_col=0).reset_index().groupby("index")["test_acc"].mean()

index
epe_nas     -0.232228
fisher       0.041336
grad_norm    0.186782
grasp        0.013587
jacob_cov   -0.303095
nwot              NaN
plain       -0.085789
snip         0.067450
Name: test_acc, dtype: float64